In [ ]:
# This line tells iPython to not display warnings.
import warnings
warnings.filterwarnings('ignore')

# RUN THIS CELL FIRST or the notebook won't work
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
import requests
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
from IPython.display import display
import matplotlib.pyplot as plt

# These help the maps display nicely in the notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = [30, 20]

In [ ]:
toronto_gpd = gpd.read_file('~/git/cp101.github.io/labs/lab10/shapefiles/toronto/toronto.shp')
toronto_gpd.head()

In [ ]:
def clean_can_census_data(can_df):
    can_df = can_df.fillna(0)
    can_df = can_df.replace({'NA': 0})
    can_df = can_df.replace({'': 0})
    can_df.iloc[:,4:] = can_df.iloc[:,4:].apply(pd.to_numeric)
    # pad to get correct geouid length
    can_df["GeoUID"] = can_df["GeoUID"].astype(str).str.ljust(10, "0")
    can_df = can_df.drop(columns = 
    return can_df

In [ ]:
lab01_data = pd.read_csv('~/git/cp101.github.io/labs/lab01/lab01_data.csv')
lab01_data = clean_can_census_data(lab01_data)
lab01_data.head()

In [ ]:
lab03_data = pd.read_csv('~/git/cp101.github.io/labs/lab03/census21_data.csv')
lab03_data = clean_can_census_data(lab03_data)
lab03_data = lab03_data.rename(columns = {'GeoUID' : 'GeoUIDX'})
lab03_data = lab03_data.drop(columns = lab03_data.columns[lab03_data.columns.isin(lab01_data.columns)])
lab03_data = lab03_data.rename(columns = {'GeoUIDX' : 'GeoUID'})
lab03_data.head()

In [ ]:
# fix missingness and data types
lab10_data = pd.read_csv('~/git/cp101.github.io/labs/lab10/census21_data.csv')
lab10_data = clean_can_census_data(lab10_data)
lab10_data = lab10_data.rename(columns = {'GeoUID' : 'GeoUIDX'})
lab10_data = lab10_data.drop(columns = lab10_data.columns[lab10_data.columns.isin(lab01_data.columns) | lab10_data.columns.isin(lab03_data.columns)])
lab10_data = lab10_data.rename(columns = {'GeoUIDX' : 'GeoUID'})
lab10_data.head()

In [111]:
# union of all columns so far in the course
all_can_census = lab01_data.merge(lab03_data, on = 'GeoUID').merge(lab10_data, on = 'GeoUID')
all_can_census.head()

,Unnamed: 0,GeoUID,Type,Region Name,Area (sq km),Population,Dwellings,Households,CMA_UID,PR_UID,...,v_CA21_1049: 18 to 64 years,v_CA21_1052: 65 years and over,"v_CA21_1085: Prevalence of low income based on the Low-income cut-offs, after tax (LICO-AT) (%)",v_CA21_1088: 0 to 17 years,v_CA21_1091: 0 to 5 years,v_CA21_1094: 18 to 64 years,v_CA21_1097: 65 years and over,v_CA21_1140: Gini index on adjusted household total income,v_CA21_1141: Gini index on adjusted household market income,v_CA21_1142: Gini index on adjusted household after-tax income
0,1,5350001.00,CT,1.0,6.8192,599,253,235,35535,0.0,...,8.0,24.0,6.0,0.0,0.0,6.0,8.0,0.360,0.426,0.309
1,2,5350002.00,CT,2.0,3.3926,604,294,284,35535,0.0,...,12.0,13.0,8.4,0.0,0.0,11.0,7.0,0.346,0.460,0.301
2,3,5350003.00,CT,3.0,0.9455,457,279,265,35535,0.0,...,12.0,12.0,8.5,18.0,0.0,8.0,0.0,0.370,0.446,0.322
3,4,5350004.00,CT,4.0,0.3404,6306,3620,3276,35535,0.0,...,20.5,36.4,14.9,10.0,8.0,16.2,10.8,0.358,0.544,0.314
4,5,5350005.00,CT,5.0,0.3764,6957,4235,3720,35535,0.0,...,15.8,45.8,12.8,10.8,7.0,13.0,14.2,0.380,0.513,0.334


In [112]:
def div_0(n,d):
    try:
        return n/d
    except:
        return 0

In [113]:
all_can_census.columns[all_can_census.columns.str.lower().str.contains("%")]

Index(['v_CA21_4289: Spending less than 30% of income on shelter costs',
       'v_CA21_4290: Spending 30% or more of income on shelter costs',
       'v_CA21_4291: 30% to less than 100%',
       'v_CA21_4293: Total - Households 'spending 30% or more of income on shelter costs' or 'not suitable' or 'major repairs needed'',
       'v_CA21_4294: Spending 30% or more of income on shelter costs only',
       'v_CA21_4297: 'Spending 30% or more of income on shelter costs' and 'not suitable'',
       'v_CA21_4298: 'Spending 30% or more of income on shelter costs' and 'major repairs needed'',
       'v_CA21_4300: 'Spending 30% or more of income on shelter costs' and 'not suitable' and 'major repairs needed'',
       'v_CA21_4302: Total - Owner and tenant households with household total income greater than zero and shelter-cost-to-income ratio less than 100%, in non-farm, non-reserve private dwellings',
       'v_CA21_4306: % of owner households with a mortgage (58)',
       'v_CA21_4307: % of

In [114]:
# create variables
all_can_census['total_pop'] = all_can_census['Population']
all_can_census['pct_rent'] = div_0(all_can_census['v_CA21_4239: Renter'] , all_can_census['v_CA21_4237: Total - Private households by tenure'])

all_can_census['pct_south_asian'] = div_0(all_can_census['v_CA21_4878: South Asian'] , all_can_census['v_CA21_4872: Total - Visible minority for the population in private households'])
all_can_census['pct_chinese'] = div_0(all_can_census['v_CA21_4881: Chinese'] , all_can_census['v_CA21_4872: Total - Visible minority for the population in private households'])
all_can_census['pct_black'] = div_0(all_can_census['v_CA21_4884: Black'] , all_can_census['v_CA21_4872: Total - Visible minority for the population in private households'])
all_can_census['pct_filipino'] = div_0(all_can_census['v_CA21_4887: Filipino'] , all_can_census['v_CA21_4872: Total - Visible minority for the population in private households'])
all_can_census['pct_arab'] = div_0(all_can_census['v_CA21_4890: Arab'] , all_can_census['v_CA21_4872: Total - Visible minority for the population in private households'])
all_can_census['pct_latin_american'] = div_0(all_can_census['v_CA21_4893: Latin American'] , all_can_census['v_CA21_4872: Total - Visible minority for the population in private households'])
all_can_census['pct_southeast_asian'] = div_0(all_can_census['v_CA21_4896: Southeast Asian'] , all_can_census['v_CA21_4872: Total - Visible minority for the population in private households'])
all_can_census['pct_west_asian'] = div_0(all_can_census['v_CA21_4899: West Asian'] , all_can_census['v_CA21_4872: Total - Visible minority for the population in private households'])
all_can_census['pct_korean'] = div_0(all_can_census['v_CA21_4902: Korean'] , all_can_census['v_CA21_4237: Total - Private households by tenure'])
all_can_census['pct_japanese'] = div_0(all_can_census['v_CA21_4905: Japanese'] , all_can_census['v_CA21_4237: Total - Private households by tenure'])
all_can_census['pct_other'] = div_0(all_can_census['v_CA21_4908: Visible minority, n.i.e.'] , all_can_census['v_CA21_4872: Total - Visible minority for the population in private households'])
all_can_census['pct_not_visible_minority'] = div_0(all_can_census['v_CA21_4914: Not a visible minority'] , all_can_census['v_CA21_4872: Total - Visible minority for the population in private households'])

all_can_census['pct_smallmf'] = div_0(all_can_census['v_CA21_438: Apartment or flat in a duplex'] , all_can_census['v_CA21_434: Occupied private dwellings by structural type of dwelling data'])
all_can_census['pct_medmf'] = div_0(all_can_census['v_CA21_439: Apartment in a building that has fewer than five storeys'] , all_can_census['v_CA21_434: Occupied private dwellings by structural type of dwelling data'])
all_can_census['pct_bigmf'] = div_0(all_can_census['v_CA21_440: Apartment in a building that has five or more storeys'] , all_can_census['v_CA21_434: Occupied private dwellings by structural type of dwelling data'])

all_can_census['pct_vacant'] = 1 - (div_0(all_can_census['v_CA21_5: Private dwellings occupied by usual residents'], all_can_census['v_CA21_4: Total private dwellings']))

all_can_census['pct_before1960'] = div_0(all_can_census['v_CA21_4264: 1960 or before'] , all_can_census['v_CA21_4263: Total - Occupied private dwellings by period of construction'])

all_can_census['pct_limat'] = all_can_census['v_CA21_1040: Prevalence of low income based on the Low-income measure, after tax (LIM-AT) (%)']
all_can_census['pct_licoat'] = all_can_census['v_CA21_1085: Prevalence of low income based on the Low-income cut-offs, after tax (LICO-AT) (%)']

all_can_census['pct_mortgage'] = all_can_census['v_CA21_4306: % of owner households with a mortgage (58)']
all_can_census['pct_owner_rent_burden'] = all_can_census['v_CA21_4307: % of owner households spending 30% or more of its income on shelter costs (55)']
all_can_census['pct_owner_ichn'] = all_can_census['v_CA21_4308: % in core housing need (57)']
all_can_census['median_monthly_shelter_owned'] = all_can_census['v_CA21_4309: Median monthly shelter costs for owned dwellings ($) (59)']
all_can_census['pct_renter_subsidized'] = all_can_census['v_CA21_4314: % of tenant households in subsidized housing (61)']
all_can_census['pct_renter_rent_burden'] = all_can_census['v_CA21_4315: % of tenant households spending 30% or more of its income on shelter costs (55)']
all_can_census['pct_renter_ichn'] = all_can_census['v_CA21_4316: % in core housing need (57)']
all_can_census['median_monthly_shelter_rented'] = all_can_census['v_CA21_4317: Median monthly shelter costs for rented dwellings ($) (59)']

subset_cols = all_can_census.iloc[:, 0:2].merge(all_can_census.iloc[:, -29:], left_index = True, right_index = True, how = 'left')
subset_cols

,Unnamed: 0,GeoUID,total_pop,pct_rent,pct_south_asian,pct_chinese,pct_black,pct_filipino,pct_arab,pct_latin_american,...,pct_limat,pct_licoat,pct_mortgage,pct_owner_rent_burden,pct_owner_ichn,median_monthly_shelter_owned,pct_renter_subsidized,pct_renter_rent_burden,pct_renter_ichn,median_monthly_shelter_rented
0,1,5350001.00,599,0.224490,0.059322,0.042373,0.000000,0.050847,0.000000,0.025424,...,9.6,6.0,73.7,18.9,10.8,2060.0,0.0,36.4,27.3,1520.0
1,2,5350002.00,604,0.105263,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,11.6,8.4,31.4,15.7,16.7,532.0,0.0,71.4,50.0,1500.0
2,3,5350003.00,457,0.314815,0.171429,0.038095,0.038095,0.038095,0.114286,0.066667,...,12.8,8.5,70.3,45.9,16.2,1800.0,0.0,17.6,0.0,2060.0
3,4,5350004.00,6306,0.926606,0.080440,0.024555,0.104996,0.055885,0.011008,0.030483,...,22.9,14.9,54.2,29.2,11.1,1640.0,9.7,44.8,44.2,1130.0
4,5,5350005.00,6957,0.736559,0.080238,0.040119,0.144874,0.043834,0.012630,0.029718,...,19.8,12.8,82.1,32.3,10.6,2220.0,25.7,36.0,30.5,1300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,581,5350378.26,4867,0.112903,0.651210,0.092742,0.067540,0.079637,0.010081,0.002016,...,4.9,3.7,75.0,27.7,5.5,2300.0,0.0,17.9,17.9,1980.0
581,582,5350378.27,5547,0.163934,0.573622,0.018067,0.119241,0.123758,0.005420,0.010840,...,5.0,3.2,82.7,25.9,8.7,2080.0,0.0,34.0,28.0,1900.0
582,583,5350378.28,6946,0.207254,0.496029,0.038267,0.182671,0.064260,0.010108,0.018051,...,7.2,4.9,60.3,20.6,8.9,1720.0,27.8,27.8,26.9,1200.0
583,584,5350802.01,4011,0.050193,0.272727,0.040260,0.080519,0.083117,0.012987,0.007792,...,5.0,3.2,55.3,16.3,6.1,1450.0,0.0,0.0,0.0,2100.0


In [ ]:
toronto_gpd.crs

In [ ]:
# reprojects coordinates to new coordinate reference system
toronto_gpd = toronto_gpd.to_crs(4326)


In [ ]:
# a lot is going on here, but it is just reading in the contents of the zipped file to dataframes
# adapted from: https://max-coding.medium.com/how-to-process-gtfs-data-using-pandas-geopandas-4b34f2ad3273

with ZipFile("../lab10/ttc/routes_schedules.zip") as myzip:
    stops_df = pd.read_csv(myzip.open("stops.txt"), dtype={
        'stop_id': 'str', 
        'stop_code': 'str',
        'stop_name': 'str',
        'stop_lat': 'float',
        'stop_lon': 'float'
    })
    stops_gdf = gpd.GeoDataFrame(stops_df, 
        geometry=gpd.points_from_xy(stops_df.stop_lon, stops_df.stop_lat)).set_crs(epsg=4326)

In [105]:
stops_gdf.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,geometry
0,262,662,Danforth Rd at Kennedy Rd,NaN,43.714379,-79.260939,NaN,NaN,NaN,NaN,NaN,1,POINT (-79.26094 43.71438)
1,263,929,Davenport Rd at Bedford Rd,NaN,43.674448,-79.399659,NaN,NaN,NaN,NaN,NaN,1,POINT (-79.39966 43.67445)
2,264,940,Davenport Rd at Dupont St,NaN,43.675511,-79.401938,NaN,NaN,NaN,NaN,NaN,2,POINT (-79.40194 43.67551)
3,265,1871,Davisville Ave at Cleveland St,NaN,43.702088,-79.378112,NaN,NaN,NaN,NaN,NaN,1,POINT (-79.37811 43.70209)
4,266,11700,Disco Rd at Attwell Dr,NaN,43.701362,-79.594843,NaN,NaN,NaN,NaN,NaN,1,POINT (-79.59484 43.70136)


In [115]:
toronto_stops = pd.DataFrame(toronto_gpd.sjoin(stops_gdf).merge(subset_cols, on = 'GeoUID', how = 'inner'))
toronto_stops.head()

,CTUID,DGUID,CTNAME,LANDAREA,PRUID,Unnamed_ 0,GeoUID,Type,Region Nam,Area (sq k,...,pct_limat,pct_licoat,pct_mortgage,pct_owner_rent_burden,pct_owner_ichn,median_monthly_shelter_owned,pct_renter_subsidized,pct_renter_rent_burden,pct_renter_ichn,median_monthly_shelter_rented
0,5350128.04,2021S05075350128.04,0128.04,0.162,35,160,5350128.04,CT,128.04,0.162,...,16.4,9.4,57.1,29.9,10.6,1840.0,21.2,43.9,27.4,1540.0
1,5350128.04,2021S05075350128.04,0128.04,0.162,35,160,5350128.04,CT,128.04,0.162,...,16.4,9.4,57.1,29.9,10.6,1840.0,21.2,43.9,27.4,1540.0
2,5350363.06,2021S05075350363.06,0363.06,0.821,35,516,5350363.06,CT,363.06,0.821,...,16.7,10.5,68.4,43.6,22.4,1760.0,7.7,40.6,25.4,1800.0
3,5350363.06,2021S05075350363.06,0363.06,0.821,35,516,5350363.06,CT,363.06,0.821,...,16.7,10.5,68.4,43.6,22.4,1760.0,7.7,40.6,25.4,1800.0
4,5350363.06,2021S05075350363.06,0363.06,0.821,35,516,5350363.06,CT,363.06,0.821,...,16.7,10.5,68.4,43.6,22.4,1760.0,7.7,40.6,25.4,1800.0


In [116]:
toronto_stops.columns

Index(['CTUID', 'DGUID', 'CTNAME', 'LANDAREA', 'PRUID', 'Unnamed_ 0', 'GeoUID',
       'Type', 'Region Nam', 'Area (sq k', 'Population', 'Dwellings',
       'Households', 'CMA_UID', 'PR_UID', 'CSD_UID', 'CD_UID', 'v_CA21_104',
       'v_CA21_1_1', 'v_CA21_1_2', 'v_CA21_1_3', 'v_CA21_105', 'v_CA21_108',
       'v_CA21_1_4', 'v_CA21_109', 'v_CA21_1_5', 'v_CA21_1_6', 'v_CA21_114',
       'v_CA21_1_7', 'v_CA21_1_8', 'v_CA21_8_', 'v_CA21_251', 'v_CA21_386',
       'v_CA21_389', 'geometry', 'index_right', 'stop_id', 'stop_code',
       'stop_name', 'stop_desc', 'stop_lat', 'stop_lon', 'zone_id', 'stop_url',
       'location_type', 'parent_station', 'stop_timezone',
       'wheelchair_boarding', 'Unnamed: 0', 'total_pop', 'pct_rent',
       'pct_south_asian', 'pct_chinese', 'pct_black', 'pct_filipino',
       'pct_arab', 'pct_latin_american', 'pct_southeast_asian',
       'pct_west_asian', 'pct_korean', 'pct_japanese', 'pct_other',
       'pct_not_visible_minority', 'pct_smallmf', 'pct_medmf'

In [117]:
toronto_stops = toronto_stops.drop(
    columns = ['CTUID', 'DGUID', 'CTNAME', 'LANDAREA', 'PRUID', 'Unnamed_ 0', 
               'Type', 'Region Nam', 'Area (sq k', 'Population', 'Dwellings',
       'Households', 'CMA_UID', 'PR_UID', 'CSD_UID', 'CD_UID', 'v_CA21_104',
       'v_CA21_1_1', 'v_CA21_1_2', 'v_CA21_1_3', 'v_CA21_105', 'v_CA21_108',
       'v_CA21_1_4', 'v_CA21_109', 'v_CA21_1_5', 'v_CA21_1_6', 'v_CA21_114',
       'v_CA21_1_7', 'v_CA21_1_8', 'v_CA21_8_', 'v_CA21_251', 'v_CA21_386',
       'v_CA21_389', 'index_right', 'stop_code',
       'stop_name', 'stop_desc', 'stop_lat', 'stop_lon', 'zone_id', 'stop_url',
       'location_type', 'parent_station', 'stop_timezone',
       'wheelchair_boarding', 'Unnamed: 0', 'geometry']
)

In [118]:
toronto_stops.head()

,GeoUID,stop_id,total_pop,pct_rent,pct_south_asian,pct_chinese,pct_black,pct_filipino,pct_arab,pct_latin_american,...,pct_limat,pct_licoat,pct_mortgage,pct_owner_rent_burden,pct_owner_ichn,median_monthly_shelter_owned,pct_renter_subsidized,pct_renter_rent_burden,pct_renter_ichn,median_monthly_shelter_rented
0,5350128.04,6440,4772,0.574290,0.092037,0.041365,0.065150,0.019648,0.013444,0.040331,...,16.4,9.4,57.1,29.9,10.6,1840.0,21.2,43.9,27.4,1540.0
1,5350128.04,1220,4772,0.574290,0.092037,0.041365,0.065150,0.019648,0.013444,0.040331,...,16.4,9.4,57.1,29.9,10.6,1840.0,21.2,43.9,27.4,1540.0
2,5350363.06,8644,7176,0.323529,0.360354,0.252044,0.076975,0.084469,0.021117,0.014986,...,16.7,10.5,68.4,43.6,22.4,1760.0,7.7,40.6,25.4,1800.0
3,5350363.06,5002,7176,0.323529,0.360354,0.252044,0.076975,0.084469,0.021117,0.014986,...,16.7,10.5,68.4,43.6,22.4,1760.0,7.7,40.6,25.4,1800.0
4,5350363.06,9289,7176,0.323529,0.360354,0.252044,0.076975,0.084469,0.021117,0.014986,...,16.7,10.5,68.4,43.6,22.4,1760.0,7.7,40.6,25.4,1800.0


In [119]:
toronto_stops.to_csv("toronto_stops.csv")